
 <h1 style="text-align: center;" class="list-group-item list-group-item-action active">Alright, lets build some nifty ML systems!</h1>

Hold up, not so fast kiddo. There are some steps we have to take before getting into predictive modeling. 

Typically, before throwing our data into a black box machine learning algorithm and praying that it works, we need to get to know our data a bit better. Specifically, we need to `understand certain statistics about the dataset` which will `help us when we get to redictive modeling`.



 <h1 style="text-align: center;" class="list-group-item list-group-item-action active">Table of Contents</h1>

<ul style="list-style-type:none;">
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#1" role="tab"
                aria-controls="settings">1. Introduction<span class="badge badge-primary badge-pill">1</span></a>
        </li>
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2" role="tab"
                aria-controls="settings">2. Handling Missing Values<span
                    class="badge badge-primary badge-pill">2</span></a>
            <ul style="list-style-type:none;">
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1" role="tab"
                        aria-controls="settings">2.1 Visualizing Missing Data <span
                            class="badge badge-primary badge-pill">3</span></a>
                    <ul style="list-style-type:none;">
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.1"
                                role="tab" aria-controls="settings">2.1.1 Matrix <span
                                    class="badge badge-primary badge-pill">4</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.2"
                                role="tab" aria-controls="settings">2.1.2 Correlation Heatmap <span
                                    class="badge badge-primary badge-pill">5</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.3"
                                role="tab" aria-controls="settings">2.1.3 Dendrogram <span
                                    class="badge badge-primary badge-pill">6</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.1.4"
                                role="tab" aria-controls="settings">2.1.4 Simple Numeric Summaries<span
                                    class="badge badge-primary badge-pill">7</span></a></li>
                    </ul>
                </li>
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2" role="tab"
                        aria-controls="settings">2.2 Method to Handle Missing Data<span
                            class="badge badge-primary badge-pill">8</span></a>
                    <ul style="list-style-type:none;">
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2.1"
                                role="tab" aria-controls="settings">2.2.1 Deletion of Data <span
                                    class="badge badge-primary badge-pill">9</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2.2"
                                role="tab" aria-controls="settings">2.2.2 Encoding Missingness<span
                                    class="badge badge-primary badge-pill">10</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#2.2.3"
                                role="tab" aria-controls="settings">2.2.3 Imputation Methods<span
                                    class="badge badge-primary badge-pill">11</span></a></li>
                    </ul>
                </li>
            </ul>
        </li>
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3" role="tab"
                aria-controls="settings">3. Encoding Categorical Attributes<span
                    class="badge badge-primary badge-pill">12</span></a>
            <ul style="list-style-type:none;">
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1" role="tab"
                        aria-controls="settings">3.1 Supervised Encoding Methods <span
                            class="badge badge-primary badge-pill">13</span></a>
                    <ul style="list-style-type:none;">
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1.1"
                                role="tab" aria-controls="settings">3.1.1 Likelihood Encoding <span
                                    class="badge badge-primary badge-pill">14</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1.2"
                                role="tab" aria-controls="settings">3.1.2 Target Encoding <span
                                    class="badge badge-primary badge-pill">15</span></a></li>
                        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.1.3"
                                role="tab" aria-controls="settings">3.1.3 Deep Learning Methods <span
                                    class="badge badge-primary badge-pill">16</span></a></li>
                    </ul>
                </li>
                <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#3.2" role="tab"
                        aria-controls="settings">3.2 Approaches for Novel Categories<span
                            class="badge badge-primary badge-pill">17</span></a>
            </ul>
        </li>
        <li><a class="list-group-item list-group-item-action" data-toggle="list" href="#4" role="tab"
            aria-controls="settings">4. References <span
                class="badge badge-primary badge-pill">18</span></a> </li>


 </ul>
